# BÁO CÁO ĐỒ ÁN MÔN HỌC MÁY
## Đề tài: Dự đoán số ngày hồi phục sau thảm họa thiên nhiên

**Sinh viên thực hiện:** Trần Minh Hiếu  
**Lớp:** [Điền mã lớp]  
**MSSV:** [Điền MSSV]

**Giảng viên hướng dẫn:** [Điền tên GVHD]

---

### 1. Giới thiệu đề tài

Đề tài sử dụng thuật toán CatBoost Regressor để dự đoán số ngày hồi phục (`recovery_days`) sau các thảm họa thiên nhiên dựa trên các đặc trưng như: quốc gia, loại thảm họa, mức độ nghiêm trọng, thiệt hại kinh tế, v.v.

### 2. Lý do chọn thuật toán CatBoost

- Xử lý tốt biến phân loại (categorical features) mà không cần mã hóa
- Hiệu suất cao với tập dữ liệu lớn
- Sử dụng kỹ thuật Ordered Boosting giúp giảm overfitting

## 3. Import thư viện

In [ ]:
# Cài đặt thư viện CatBoost (chỉ cần chạy 1 lần)
%pip install catboost -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from catboost import CatBoostRegressor, Pool, cv

import warnings
warnings.filterwarnings('ignore')

print("Import thư viện thành công!")

Import thư viện thành công!


## 4. Đọc và khám phá dữ liệu

In [ ]:
# Đọc file dữ liệu
df = pd.read_csv('../data/global_disaster_response_2018_2024.csv')

# Xem thông tin cơ bản
print("Kích thước dữ liệu:", df.shape)
print("\nDanh sách các cột:")
print(df.columns.tolist())
print("\n5 dòng đầu tiên:")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/global_disaster_response_2018_2024.csv'

In [ ]:
# Xem thông tin kiểu dữ liệu
print("Thông tin chi tiết:")
df.info()

print("\nThống kê mô tả:")
df.describe().round(2)

In [ ]:
# Kiểm tra dữ liệu thiếu (missing values)
print("Kiểm tra giá trị thiếu:")
missing = df.isnull().sum()
print(missing)

if missing.sum() == 0:
    print("\n=> Dữ liệu không có giá trị thiếu")

## 5. Tiền xử lý dữ liệu (Data Preprocessing)

### 5.1. Xử lý biến thời gian

In [ ]:
# Tạo bản sao dữ liệu để xử lý
df_processed = df.copy()

# Chuyển cột date sang kiểu datetime
df_processed['date'] = pd.to_datetime(df_processed['date'], format='%d/%m/%Y')

# Trích xuất năm và tháng từ cột date
df_processed['year'] = df_processed['date'].dt.year
df_processed['month'] = df_processed['date'].dt.month

print("Đã xử lý biến thời gian:")
print("- Năm:", df_processed['year'].min(), "-", df_processed['year'].max())
print("- Tháng: 1 - 12")

### 5.2. Xử lý biến phân loại (Categorical Features)

CatBoost có khả năng xử lý trực tiếp biến phân loại mà không cần One-Hot Encoding hay Label Encoding.

In [ ]:
# Xác định các biến phân loại
categorical_features = ['country', 'disaster_type']

# Xem số lượng giá trị unique của mỗi biến
for col in categorical_features:
    print(f"{col}:")
    print(f"  - Số giá trị unique: {df_processed[col].nunique()}")
    print(f"  - Các giá trị: {df_processed[col].unique().tolist()}")

### 5.3. Xử lý biến số (Log Transformation)

Áp dụng phép biến đổi log cho các biến có phân phối lệch (skewed distribution) để cải thiện hiệu suất mô hình.

In [ ]:
# Các biến cần log transform
log_transform_cols = ['economic_loss_usd', 'aid_amount_usd']

print("Log transformation:")
for col in log_transform_cols:
    # Tính skewness trước và sau khi transform
    skew_before = df_processed[col].skew()
    df_processed[f'{col}_log'] = np.log1p(df_processed[col])
    skew_after = df_processed[f'{col}_log'].skew()
    
    print(f"\n{col}:")
    print(f"  - Skewness trước: {skew_before:.3f}")
    print(f"  - Skewness sau: {skew_after:.3f}")

In [ ]:
# Biểu đồ so sánh phân phối trước và sau log transform
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for idx, col in enumerate(log_transform_cols):
    # Phân phối gốc
    axes[idx, 0].hist(df_processed[col], bins=50, color='blue', alpha=0.7, edgecolor='black')
    axes[idx, 0].set_title(f'Truoc transform: {col}')
    axes[idx, 0].set_xlabel(col)
    axes[idx, 0].set_ylabel('Tan suat')
    
    # Phân phối sau log transform
    axes[idx, 1].hist(df_processed[f'{col}_log'], bins=50, color='orange', alpha=0.7, edgecolor='black')
    axes[idx, 1].set_title(f'Sau log transform: {col}_log')
    axes[idx, 1].set_xlabel(f'{col}_log')
    axes[idx, 1].set_ylabel('Tan suat')

plt.tight_layout()
plt.savefig('../models/log_transform_comparison.png', dpi=150)
plt.show()

### 5.4. Feature Engineering

Tạo thêm các đặc trưng mới từ các biến có sẵn để cải thiện khả năng dự đoán của mô hình.

In [ ]:
# Tạo các đặc trưng mới
print("Tạo các đặc trưng mới:")

# 1. Thiệt hại kinh tế trên mỗi thương vong
df_processed['loss_per_casualty'] = df_processed['economic_loss_usd'] / (df_processed['casualties'] + 1)
print("1. loss_per_casualty = economic_loss_usd / (casualties + 1)")

# 2. Tiền viện trợ trên mỗi giờ phản ứng
df_processed['aid_per_hour'] = df_processed['aid_amount_usd'] / (df_processed['response_time_hours'] + 1)
print("2. aid_per_hour = aid_amount_usd / (response_time_hours + 1)")

# 3. Tỷ lệ mức độ nghiêm trọng / hiệu quả phản ứng
df_processed['severity_response_ratio'] = df_processed['severity_index'] / (df_processed['response_efficiency_score'] + 1) * 100
print("3. severity_response_ratio = severity_index / response_efficiency_score * 100")

# 4. Hiệu quả viện trợ
df_processed['aid_efficiency'] = df_processed['aid_amount_usd'] / (df_processed['economic_loss_usd'] + 1)
print("4. aid_efficiency = aid_amount_usd / economic_loss_usd")

print("\n=> Đã tạo 4 đặc trưng mới")

## 6. Phân tích biến mục tiêu (Target Variable)

Biến mục tiêu cần dự đoán là `recovery_days` - số ngày hồi phục sau thảm họa.

In [ ]:
# Biểu đồ phân phối biến mục tiêu
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Histogram
axes[0].hist(df_processed['recovery_days'], bins=50, color='steelblue', edgecolor='black')
axes[0].axvline(df_processed['recovery_days'].mean(), color='red', linestyle='--', label='Mean')
axes[0].set_xlabel('So ngay hoi phuc')
axes[0].set_ylabel('Tan suat')
axes[0].set_title('Phan phoi Recovery Days')
axes[0].legend()

# Box plot
axes[1].boxplot(df_processed['recovery_days'])
axes[1].set_ylabel('So ngay hoi phuc')
axes[1].set_title('Box Plot')

# Density plot
df_processed['recovery_days'].plot(kind='kde', ax=axes[2], color='steelblue')
axes[2].set_xlabel('So ngay hoi phuc')
axes[2].set_title('Mat do phan phoi')

plt.tight_layout()
plt.savefig('../models/target_distribution.png', dpi=150)
plt.show()

# Thống kê mô tả
print("Thong ke bien muc tieu recovery_days:")
print(f"  - Trung binh: {df_processed['recovery_days'].mean():.2f} ngay")
print(f"  - Trung vi: {df_processed['recovery_days'].median():.2f} ngay")
print(f"  - Do lech chuan: {df_processed['recovery_days'].std():.2f} ngay")
print(f"  - Min: {df_processed['recovery_days'].min()} ngay")
print(f"  - Max: {df_processed['recovery_days'].max()} ngay")

## 7. Chuẩn bị dữ liệu cho mô hình

In [ ]:
# Danh sách các đặc trưng (features) sử dụng cho mô hình
feature_cols = [
    # Biến phân loại
    'country', 'disaster_type',
    
    # Biến số gốc
    'severity_index', 'casualties', 'response_time_hours', 
    'response_efficiency_score', 'latitude', 'longitude',
    
    # Biến đã log transform
    'economic_loss_usd_log', 'aid_amount_usd_log',
    
    # Biến thời gian
    'year', 'month',
    
    # Đặc trưng mới
    'loss_per_casualty', 'aid_per_hour', 'severity_response_ratio', 'aid_efficiency'
]

# Biến mục tiêu
target_col = 'recovery_days'

print(f"Tong so features: {len(feature_cols)}")
print("\nDanh sach features:")
for i, col in enumerate(feature_cols):
    ftype = "(categorical)" if col in categorical_features else "(numerical)"
    print(f"  {i+1}. {col} {ftype}")

In [ ]:
# Tách features (X) và target (y)
X = df_processed[feature_cols].copy()
y = df_processed[target_col].copy()

# Chia dữ liệu thành tập train và test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Chia du lieu:")
print(f"  - Tap train: {X_train.shape[0]} mau ({X_train.shape[0]/len(X)*100:.0f}%)")
print(f"  - Tap test: {X_test.shape[0]} mau ({X_test.shape[0]/len(X)*100:.0f}%)")

In [ ]:
# Tạo CatBoost Pool (định dạng dữ liệu riêng của CatBoost)
train_pool = Pool(
    data=X_train, 
    label=y_train, 
    cat_features=categorical_features
)

test_pool = Pool(
    data=X_test, 
    label=y_test, 
    cat_features=categorical_features
)

print("Da tao CatBoost Pool voi categorical features")

## 8. Xây dựng và huấn luyện mô hình CatBoost

In [ ]:
# Khởi tạo mô hình CatBoost Regressor
model = CatBoostRegressor(
    iterations=1000,           # Số lượng cây tối đa
    learning_rate=0.05,        # Tốc độ học
    depth=8,                   # Độ sâu cây
    l2_leaf_reg=3,             # Hệ số regularization L2
    min_data_in_leaf=20,       # Số mẫu tối thiểu trong leaf node
    loss_function='RMSE',      # Hàm mất mát
    eval_metric='RMSE',        # Metric đánh giá
    random_seed=42,
    verbose=100                # In log mỗi 100 iterations
)

print("Cac tham so mo hinh:")
print("  - iterations: 1000")
print("  - learning_rate: 0.05")
print("  - depth: 8")
print("  - l2_leaf_reg: 3")
print("  - loss_function: RMSE")

In [ ]:
# Huấn luyện mô hình với early stopping
print("🚀 Bắt đầu huấn luyện CatBoost...\n")

model.fit(
    train_pool,
    eval_set=test_pool,
    early_stopping_rounds=50,  # Dừng nếu không cải thiện sau 50 rounds
    plot=False
)

print(f"\n✅ Hoàn thành huấn luyện!")
print(f"   - Best iteration: {model.best_iteration_}")
print(f"   - Best RMSE: {model.best_score_['validation']['RMSE']:.4f}")

## 7. Đánh giá mô hình

In [ ]:
# Dự đoán
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Hàm tính các metrics
def evaluate_metrics(y_true, y_pred, dataset_name):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n📊 {dataset_name}:")
    print(f"   - MAE  (Mean Absolute Error): {mae:.4f} ngày")
    print(f"   - RMSE (Root Mean Squared Error): {rmse:.4f} ngày")
    print(f"   - R²   (R-squared Score): {r2:.4f}")
    
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

print("="*60)
print("📈 KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH CATBOOST")
print("="*60)

train_metrics = evaluate_metrics(y_train, y_train_pred, "TẬP HUẤN LUYỆN (Train)")
test_metrics = evaluate_metrics(y_test, y_test_pred, "TẬP KIỂM TRA (Test)")

print("\n" + "="*60)
print(f"\n🎯 Kết luận:")
print(f"   - Mô hình dự đoán sai trung bình {test_metrics['MAE']:.2f} ngày")
print(f"   - Mô hình giải thích được {test_metrics['R2']*100:.2f}% phương sai của dữ liệu")

In [ ]:
# Visualization: Actual vs Predicted
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Scatter plot: Actual vs Predicted
axes[0].scatter(y_test, y_test_pred, alpha=0.3, color='steelblue', s=10)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect prediction')
axes[0].set_xlabel('Giá trị thực tế (ngày)', fontsize=12)
axes[0].set_ylabel('Giá trị dự đoán (ngày)', fontsize=12)
axes[0].set_title('Actual vs Predicted', fontsize=14)
axes[0].legend()

# 2. Residual plot
residuals = y_test - y_test_pred
axes[1].scatter(y_test_pred, residuals, alpha=0.3, color='coral', s=10)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Giá trị dự đoán (ngày)', fontsize=12)
axes[1].set_ylabel('Residuals (ngày)', fontsize=12)
axes[1].set_title('Residual Plot', fontsize=14)

# 3. Distribution of residuals
axes[2].hist(residuals, bins=50, edgecolor='black', color='green', alpha=0.7)
axes[2].axvline(x=0, color='r', linestyle='--', lw=2)
axes[2].set_xlabel('Residuals (ngày)', fontsize=12)
axes[2].set_ylabel('Frequency', fontsize=12)
axes[2].set_title(f'Distribution of Residuals\nMean: {residuals.mean():.2f}, Std: {residuals.std():.2f}', fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
# Feature Importance từ CatBoost
feature_importance = model.get_feature_importance(prettified=True)

plt.figure(figsize=(12, 10))
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(feature_importance)))
plt.barh(feature_importance['Feature Id'], feature_importance['Importances'], color=colors)
plt.xlabel('Importance (%)', fontsize=12)
plt.title('🎯 Feature Importance - CatBoost', fontsize=14)
plt.gca().invert_yaxis()  # Quan trọng nhất ở trên
plt.tight_layout()
plt.show()

print("\n📊 Top 10 đặc trưng quan trọng nhất:")
print(feature_importance.head(10).to_string(index=False))

## 8. Cross-Validation

In [ ]:
# Cross-validation với CatBoost
from catboost import cv

# Tạo Pool từ toàn bộ dữ liệu
full_pool = Pool(data=X, label=y, cat_features=categorical_features)

# Thực hiện 5-fold cross-validation
print("🔄 Thực hiện 5-Fold Cross-Validation...\n")

cv_params = model.get_params()
cv_params['iterations'] = model.best_iteration_  # Sử dụng best iteration

cv_results = cv(
    pool=full_pool,
    params=cv_params,
    fold_count=5,
    shuffle=True,
    partition_random_seed=42,
    verbose=False
)

print("\n📊 KẾT QUẢ CROSS-VALIDATION (5-Fold):")
print("="*50)
final_rmse_mean = cv_results['test-RMSE-mean'].iloc[-1]
final_rmse_std = cv_results['test-RMSE-std'].iloc[-1]
print(f"   - RMSE: {final_rmse_mean:.4f} ± {final_rmse_std:.4f}")
print("="*50)

## 9. Hệ thống dự đoán số ngày hồi phục

In [ ]:
# Class dự đoán số ngày hồi phục
class RecoveryDaysPredictor:
    def __init__(self, model, feature_cols, categorical_features):
        self.model = model
        self.feature_cols = feature_cols
        self.categorical_features = categorical_features
    
    def preprocess(self, disaster_data):
        """
        Tiền xử lý dữ liệu đầu vào.
        """
        df_input = pd.DataFrame([disaster_data])
        
        # Log-transform
        df_input['economic_loss_usd_log'] = np.log1p(df_input['economic_loss_usd'])
        df_input['aid_amount_usd_log'] = np.log1p(df_input['aid_amount_usd'])
        
        # Feature engineering
        df_input['loss_per_casualty'] = df_input['economic_loss_usd'] / (df_input['casualties'] + 1)
        df_input['aid_per_hour'] = df_input['aid_amount_usd'] / (df_input['response_time_hours'] + 1)
        df_input['severity_response_ratio'] = df_input['severity_index'] / (df_input['response_efficiency_score'] + 1) * 100
        df_input['aid_efficiency'] = df_input['aid_amount_usd'] / (df_input['economic_loss_usd'] + 1)
        
        return df_input[self.feature_cols]
    
    def predict(self, disaster_data):
        """
        Dự đoán số ngày hồi phục.
        """
        X_input = self.preprocess(disaster_data)
        prediction = self.model.predict(X_input)[0]
        return max(1, round(prediction, 1))  # Tối thiểu 1 ngày

# Khởi tạo predictor
predictor = RecoveryDaysPredictor(model, feature_cols, categorical_features)
print("✅ Hệ thống dự đoán đã sẵn sàng!")

In [ ]:
# Các quốc gia và loại thảm họa có trong dữ liệu
print("🌍 Danh sách quốc gia có trong dữ liệu:")
print(sorted(df['country'].unique()))
print("\n🌋 Danh sách loại thảm họa:")
print(sorted(df['disaster_type'].unique()))

In [ ]:
# Ví dụ 1: Động đất tại Japan
disaster_example_1 = {
    'country': 'Japan',
    'disaster_type': 'Earthquake',
    'severity_index': 7.5,
    'casualties': 150,
    'economic_loss_usd': 5000000,
    'response_time_hours': 8,
    'aid_amount_usd': 300000,
    'response_efficiency_score': 85,
    'latitude': 35.6762,
    'longitude': 139.6503,
    'year': 2024,
    'month': 6
}

prediction_1 = predictor.predict(disaster_example_1)
print("🌏 VÍ DỤ 1: Động đất tại Japan")
print(f"   - Mức độ nghiêm trọng: {disaster_example_1['severity_index']}")
print(f"   - Số thương vong: {disaster_example_1['casualties']}")
print(f"   - Thiệt hại kinh tế: ${disaster_example_1['economic_loss_usd']:,}")
print(f"   - Thời gian phản ứng: {disaster_example_1['response_time_hours']} giờ")
print(f"   📌 DỰ ĐOÁN SỐ NGÀY HỒI PHỤC: {prediction_1} ngày")

In [ ]:
# Ví dụ 2: Bão tại Philippines
disaster_example_2 = {
    'country': 'Philippines',
    'disaster_type': 'Hurricane',
    'severity_index': 8.2,
    'casualties': 200,
    'economic_loss_usd': 8000000,
    'response_time_hours': 12,
    'aid_amount_usd': 500000,
    'response_efficiency_score': 75,
    'latitude': 14.5995,
    'longitude': 120.9842,
    'year': 2024,
    'month': 9
}

prediction_2 = predictor.predict(disaster_example_2)
print("🌀 VÍ DỤ 2: Bão tại Philippines")
print(f"   - Mức độ nghiêm trọng: {disaster_example_2['severity_index']}")
print(f"   - Số thương vong: {disaster_example_2['casualties']}")
print(f"   - Thiệt hại kinh tế: ${disaster_example_2['economic_loss_usd']:,}")
print(f"   - Thời gian phản ứng: {disaster_example_2['response_time_hours']} giờ")
print(f"   📌 DỰ ĐOÁN SỐ NGÀY HỒI PHỤC: {prediction_2} ngày")

In [ ]:
# Ví dụ 3: Lũ lụt tại India
disaster_example_3 = {
    'country': 'India',
    'disaster_type': 'Flood',
    'severity_index': 5.5,
    'casualties': 80,
    'economic_loss_usd': 3000000,
    'response_time_hours': 15,
    'aid_amount_usd': 200000,
    'response_efficiency_score': 70,
    'latitude': 20.5937,
    'longitude': 78.9629,
    'year': 2024,
    'month': 7
}

prediction_3 = predictor.predict(disaster_example_3)
print("🌊 VÍ DỤ 3: Lũ lụt tại India")
print(f"   - Mức độ nghiêm trọng: {disaster_example_3['severity_index']}")
print(f"   - Số thương vong: {disaster_example_3['casualties']}")
print(f"   - Thiệt hại kinh tế: ${disaster_example_3['economic_loss_usd']:,}")
print(f"   - Thời gian phản ứng: {disaster_example_3['response_time_hours']} giờ")
print(f"   📌 DỰ ĐOÁN SỐ NGÀY HỒI PHỤC: {prediction_3} ngày")

## 10. Dự đoán tùy chỉnh - Nhập dữ liệu của bạn

In [ ]:
# ============================================
# 📝 NHẬP THÔNG TIN THẢM HỌA MỚI TẠI ĐÂY
# ============================================

my_disaster = {
    'country': 'Brazil',            # Quốc gia
    'disaster_type': 'Flood',       # Loại thảm họa
    'severity_index': 6.5,          # Mức độ nghiêm trọng (0-10)
    'casualties': 120,              # Số thương vong
    'economic_loss_usd': 5500000,   # Thiệt hại kinh tế (USD)
    'response_time_hours': 10,      # Thời gian phản ứng (giờ)
    'aid_amount_usd': 350000,       # Số tiền viện trợ (USD)
    'response_efficiency_score': 78, # Điểm hiệu quả phản ứng (0-100)
    'latitude': -15.7801,           # Vĩ độ
    'longitude': -47.9292,          # Kinh độ
    'year': 2024,                   # Năm
    'month': 11                     # Tháng
}

# Thực hiện dự đoán
my_prediction = predictor.predict(my_disaster)

print("\n" + "="*60)
print("📋 KẾT QUẢ DỰ ĐOÁN")
print("="*60)
print(f"🌍 Quốc gia: {my_disaster['country']}")
print(f"🌋 Loại thảm họa: {my_disaster['disaster_type']}")
print(f"📊 Mức độ nghiêm trọng: {my_disaster['severity_index']}/10")
print(f"💀 Số thương vong: {my_disaster['casualties']:,}")
print(f"💰 Thiệt hại kinh tế: ${my_disaster['economic_loss_usd']:,}")
print(f"⏱️ Thời gian phản ứng: {my_disaster['response_time_hours']} giờ")
print(f"🆘 Tiền viện trợ: ${my_disaster['aid_amount_usd']:,}")
print(f"📈 Hiệu quả phản ứng: {my_disaster['response_efficiency_score']}/100")
print("="*60)
print(f"\n🎯 DỰ ĐOÁN SỐ NGÀY HỒI PHỤC: {my_prediction} NGÀY")
print("="*60)

## 11. Lưu mô hình

In [ ]:
import os

# Tạo thư mục models nếu chưa có
os.makedirs('../models', exist_ok=True)

# Lưu mô hình CatBoost
model.save_model('../models/catboost_recovery_days.cbm')

# Lưu feature list
import json
config = {
    'feature_cols': feature_cols,
    'categorical_features': categorical_features,
    'train_metrics': train_metrics,
    'test_metrics': test_metrics
}

with open('../models/catboost_config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Đã lưu mô hình tại:")
print("   - ../models/catboost_recovery_days.cbm")
print("   - ../models/catboost_config.json")

In [ ]:
# Cách tải lại mô hình để sử dụng
print("📌 Code để tải lại mô hình:")
print("""
from catboost import CatBoostRegressor
import json

# Tải mô hình
model = CatBoostRegressor()
model.load_model('../models/catboost_recovery_days.cbm')

# Tải config
with open('../models/catboost_config.json', 'r') as f:
    config = json.load(f)

feature_cols = config['feature_cols']
categorical_features = config['categorical_features']
""")

## 12. Tổng kết

### 📊 Kết quả đạt được:
- Đã xây dựng mô hình CatBoost Regressor để dự đoán số ngày hồi phục sau thảm họa
- Dataset: 50,000 bản ghi với 11 features gốc + 4 features mới
- CatBoost xử lý trực tiếp categorical features (country, disaster_type)

### 🔧 Tiền xử lý dữ liệu:
1. **Biến thời gian:** Trích xuất year, month từ date
2. **Biến phân loại:** CatBoost xử lý trực tiếp
3. **Biến số:** Log-transform cho economic_loss_usd và aid_amount_usd
4. **Feature Engineering:**
   - loss_per_casualty
   - aid_per_hour
   - severity_response_ratio
   - aid_efficiency

### 📈 Metrics đánh giá:
- **MAE:** Sai số tuyệt đối trung bình
- **RMSE:** Căn bậc hai sai số bình phương trung bình
- **R²:** Tỉ lệ phương sai được giải thích

### 🎯 Kết luận:
CatBoost cho kết quả tốt nhờ:
- Xử lý hiệu quả categorical features
- Ordered boosting giảm overfitting
- Tối ưu cho dataset lớn